In [ ]:
import imagej
import scyjava as sj

ij = imagej.init('sc.fiji:fiji', mode='interactive', add_legacy=True)

In [ ]:
print(f"ImageJ version: {ij.getVersion()}")

In [ ]:
script = """
importClass(Packages.ij.IJ)
importClass(Packages.ij.Prefs)

#@ String file_path
#@ String whiteimage_output
#@ String skeleton_output
#@ String branch_output
#@ String summary_output

imp = IJ.openImage(file_path);
IJ.run(imp, "Enhance Contrast", "saturated=0.35")
IJ.run(imp, "Apply LUT", "");
IJ.run(imp, "Unsharp Mask...", "radius=1 mask=0.60");
IJ.run(imp, "Maximum...", "radius=0.5");
IJ.run(imp, "Despeckle", "");
IJ.run(imp, "Unsharp Mask...", "radius=1 mask=0.60");
IJ.run(imp, "Despeckle", "");
IJ.setAutoThreshold(imp, "Default dark no-reset");
IJ.setRawThreshold(imp, 100, 255);
Prefs.blackBackground = true;
IJ.run(imp, "Convert to Mask", "");
IJ.run(imp, "Close-", "");
IJ.run(imp, "Despeckle", "");
IJ.run(imp, "Remove Outliers...", "radius=4 threshold=50 which=Bright");
IJ.saveAs(imp, "Tiff", whiteimage_output);
IJ.run(imp, "Skeletonize", "");
IJ.saveAs(imp, "Tiff", skeleton_output);
IJ.run(imp, "Analyze Skeleton (2D/3D)", "prune=none calculate show");
IJ.saveAs("Results", branch_output);
IJ.run(imp, "Close", "");
IJ.saveAs("Results", summary_output);
IJ.run(imp, "Close", "");
"""

In [ ]:
import os
import time
from datetime import datetime
import warnings

# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') 

#data_path = "spine_analysis_2024/Spine density REPL1/"
#data_path = "spine_analysis_2024/Spine density REPL2/"
data_path = "spine_analysis_2024/Spine density REPL3/"

#output_base_path = "spine_analysis_2024/spine_density_output1/"
#output_base_path = "spine_analysis_2024/spine_density_output2/"
output_base_path = "spine_analysis_2024/spine_density_output3/"

def analyse_skeleton(data_path, data, output):

    # record pipeline execution time
    start_time = time.time()

    print(f"Started processing {data}")

    # Reconstruct full file path
    file_path = f"{data_path}{data}"

    # Extract the image name before .tif
    base_name = data.split(".tif")[0]
    # Generate the desired output paths
    whiteimage_output = f"{output}{base_name}_white_image.tif"
    skeleton_output = f"{output}{base_name}_skeleton.tif"
    branch_output = f"{output}{base_name}_branch_information.csv"
    summary_output = f"{output}{base_name}_summary.csv"

    args = {
        'file_path':file_path,
        'whiteimage_output':whiteimage_output,
        'skeleton_output':skeleton_output,
        'branch_output':branch_output,
        'summary_output':summary_output
    }

    ij.py.run_script("javascript", script, args)

    ij.WindowManager.closeAllWindows()

    # Display data processing pipeline metrics

    end_time = time.time()
    duration = end_time - start_time
    print(f"Finished processing {data} at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Execution time: {duration:.2f} seconds")

In [ ]:
for file in os.listdir(data_path):

    analyse_skeleton(data_path, file, output_base_path)

In [ ]:
modulenames = ['imagej','scyjava']
for module in modulenames:
    try:
        version = eval(module).__version__
        print(f"{module}=={version}")
    except:
        print(f"{module}")